In [1]:
from tensorflow.keras.optimizers import AdamW
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

2024-07-09 21:55:58.497236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 21:55:58.527839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 21:55:58.527869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 21:55:58.548009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 21:55:59.415179: W tensorflow/compiler/tf

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
import tensorflow as tf
import torch
label_encoder = LabelEncoder()

In [ ]:
df = pd.read_csv(f'/home/rpierson/Topic_Files/topic_0.csv')
df['Priority'] = label_encoder.fit_transform(df['Priority'])

In [ ]:
df.head()

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
def tokenize_data(texts, labels, tokenizer, max_len):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_len)
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    ))


In [ ]:
train_dataset = tokenize_data(train_df.Combined_Text, train_df.Priority, tokenizer, max_len=128)
val_dataset = tokenize_data(val_df.Combined_Text, val_df.Priority, tokenizer, max_len=128)


In [ ]:
train_dataset = train_dataset.shuffle(len(train_df)).batch(16)
val_dataset = val_dataset.batch(16)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [ ]:

if torch.cuda.is_available():
        print("LOG: CUDA is available. Torch is using GPU:",
                f"{torch.cuda.get_device_name(torch.cuda.current_device())}",
                flush=True)
        torch.cuda.set_device(torch.device("cuda:5"))
else:
    print("ERROR: CUDA is not available. Torch is using CPU.", flush=True)


In [ ]:
# TensorFlow GPU configuration
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print(f"LOG: TensorFlow GPU devices: {physical_devices}.", flush=True)
    except RuntimeError as exception:
        print(f"LOG: TensorFlow GPU devices: {physical_devices}.", flush=True)
        print(f"LOG: TensorFlow GPU configuration error: {exception}", flush=True)
else:
    print("ERROR: No TensorFlow GPU devices found.", flush=True)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [ ]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15
)